In [2]:
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import ResNet50
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D, Dropout
from tensorflow.keras.optimizers import Adam

In [3]:
IMG_SIZE = (224, 224)
BATCH_SIZE = 32
DATASET_DIR = "../src/data/"

datagen = ImageDataGenerator(rescale=1./255)

train_generator = datagen.flow_from_directory(
    f"{DATASET_DIR}/train", target_size=IMG_SIZE, batch_size=BATCH_SIZE, class_mode="categorical")

val_generator = datagen.flow_from_directory(
    f"{DATASET_DIR}/val", target_size=IMG_SIZE, batch_size=BATCH_SIZE, class_mode="categorical")

test_generator = datagen.flow_from_directory(
    f"{DATASET_DIR}/test", target_size=IMG_SIZE, batch_size=BATCH_SIZE, class_mode="categorical")


Found 4359 images belonging to 5 classes.
Found 932 images belonging to 5 classes.
Found 939 images belonging to 5 classes.


In [ ]:
"""base_model = ResNet50(weights='imagenet', include_top=False, input_shape=(224, 224, 3))
base_model.trainable = False  # On gèle les couches de ResNet

x = base_model.output
x = GlobalAveragePooling2D()(x)
x = Dropout(0.5)(x)
predictions = Dense(5, activation='softmax')(x)  # 5 classes

model = Model(inputs=base_model.input, outputs=predictions)"""


In [4]:
base_model_1 = ResNet50(include_top=False, weights='imagenet', input_shape=(224, 224, 3))
base_model_1.trainable = False  # on fige les couches

model_1 = tf.keras.Sequential([
    base_model_1,
    tf.keras.layers.GlobalAveragePooling2D(),
    tf.keras.layers.Dense(512, activation='relu'),
    tf.keras.layers.Dropout(0.3),
    tf.keras.layers.Dense(5, activation='softmax')
])


In [ ]:
model_1.compile(optimizer=Adam(learning_rate=0.0001), loss='categorical_crossentropy', metrics=['accuracy'])

history = model_1.fit(
    train_generator,
    epochs=10,
    validation_data=val_generator
)


Epoch 1/10
137/137 [==============================] - 1008s 7s/step - loss: 1.5169 - accuracy: 0.3207 - val_loss: 1.4083 - val_accuracy: 0.3680
Epoch 2/10
137/137 [==============================] - 803s 6s/step - loss: 1.4496 - accuracy: 0.3437 - val_loss: 1.3999 - val_accuracy: 0.3637
Epoch 3/10
137/137 [==============================] - 793s 6s/step - loss: 1.4286 - accuracy: 0.3517 - val_loss: 1.3939 - val_accuracy: 0.4099
Epoch 4/10
137/137 [==============================] - 837s 6s/step - loss: 1.4132 - accuracy: 0.3675 - val_loss: 1.3822 - val_accuracy: 0.3938
Epoch 5/10
137/137 [==============================] - 946s 7s/step - loss: 1.4016 - accuracy: 0.3820 - val_loss: 1.3768 - val_accuracy: 0.3916
Epoch 6/10
137/137 [==============================] - 951s 7s/step - loss: 1.3912 - accuracy: 0.3845 - val_loss: 1.3768 - val_accuracy: 0.4077
Epoch 7/10
137/137 [==============================] - 1003s 7s/step - loss: 1.3823 - accuracy: 0.3960 - val_loss: 1.3695 - val_accuracy: 0.40

In [6]:
test_loss, test_acc = model.evaluate(test_generator)
print(f'Précision sur le jeu de test : {test_acc*100:.2f}%')

25/30 [========================>.....] - ETA: 28s - loss: 1.4165 - accuracy: 0.3925

: 

In [ ]:
import numpy as np
from sklearn.metrics import classification_report, confusion_matrix
import seaborn as sns
import matplotlib.pyplot as plt

In [ ]:
# Prédictions
y_pred = model.predict(test_generator)
y_pred_classes = np.argmax(y_pred, axis=1)

In [ ]:
# Vraies étiquettes
y_true = test_generator.classes
class_labels = list(test_generator.class_indices.keys())

In [ ]:
report = classification_report(y_true, y_pred_classes, target_names=class_labels)
print(report)

In [ ]:
cm = confusion_matrix(y_true, y_pred_classes)

plt.figure(figsize=(8, 6))
sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', xticklabels=class_labels, yticklabels=class_labels)
plt.xlabel('Prédit')
plt.ylabel('Réel')
plt.title('Matrice de confusion')
plt.show()

In [ ]:
plt.plot(history.history['accuracy'], label='Train Accuracy')
plt.plot(history.history['val_accuracy'], label='Val Accuracy')
plt.xlabel('Epochs')
plt.ylabel('Accuracy')
plt.legend()
plt.title('Courbe d’Accuracy')
plt.show()

In [ ]:
model.save('skin_disease_classifier.h5')